属性查询的优先级
1. getattribute()， 无条件调用
2. 数据描述符：由1. 触发调用 （若人为的重载了该 getattribute() 方法，可能会导致无法调用描述符）
3. 实例对象的字典（若与描述符对象同名，会被覆盖哦）
4. 类的字典
5. 非数据描述符
6. 父类的字典
7. getattr() 方法

In [3]:
import pprint
pp = pprint.PrettyPrinter(indent=4, compact=False).pprint
import attrs
from attrs import define, field

In [4]:
import pprint
pp = pprint.PrettyPrinter(indent=4, compact=False).pprint
from attrs import define, field

class Test(object):
    cls_val = 1
    def __init__(self):
        self.ins_val = 10

t = Test()
# pp.pprint(t.__dict__)
# pp.pprint(Test.__dict__)
pp(t.__dict__)
pp(Test.__dict__)

{'ins_val': 10}
mappingproxy({   '__dict__': <attribute '__dict__' of 'Test' objects>,
                 '__doc__': None,
                 '__init__': <function Test.__init__ at 0x000001D1969EE550>,
                 '__module__': '__main__',
                 '__weakref__': <attribute '__weakref__' of 'Test' objects>,
                 'cls_val': 1})


In [5]:
class Student:
    def __init__(self, name, math, chinese, english):
        self.name = name
        self.math = math
        self.chinese = chinese
        self.english = english
    @property
    def math(self):
        return self._math
    @math.setter
    def math(self, value):
        if 0 <= value <= 100:
            self._math = value
        else:
            raise ValueError('math score must be between 0 and 100')
    @property
    def chinese(self):
        return self._chinese
    @chinese.setter
    def chinese(self, value):
        if 0 <= value <= 100:
            self._chinese = value
        else:
            raise ValueError('chinese score must be between 0 and 100')
    @property
    def english(self):
        return self._english
    @english.setter
    def english(self, value):
        if 0 <= value <= 100:
            self._english = value
        else:
            raise ValueError('english score must be between 0 and 100')
    def __repr__(self):
        return f"Student: {self.name}, math: {self.math}, chinese: {self.chinese}, english: {self.english}"

stu = Student('Tom', 90, 80, 70)
pp(stu)
print(stu)

Student: Tom, math: 90, chinese: 80, english: 70
Student: Tom, math: 90, chinese: 80, english: 70


In [6]:
@define(slots=False)
class Score:
    def __init__(self, score=0, subject=None) -> None:
        self._score = score
        self._subject = subject
    def __get__(self, instance, owner):
        return self._scocre
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise ValueError(f'{self._subject} score must be int')
        if not 0 <= value <= 100:
            raise ValueError(f'{self._subject} score must be between 0 and 100')
        self._scocre = value
    def __delete__(self):
        del self._scocre

@define(slots=False)
class Student:
    math = Score(score=0, subject='math')
    chinese = Score(score=0, subject='chinese')
    english = Score(score=0, subject='english')
    
    def __attrs_pre_init__(self):
        print('attrs_pre_init')


    """
    attrs: custom init
    
    """ 
    name: str
    math: int
    chinese: int
    english: int

    def __init__(self, name, math, chinese, english):
        self.__attrs_init__(name, math, chinese, english)
        print("custome init")

    """
    attrs: post_init
    """
    # name: str = field()
    # math: int = field()
    # chinese: int = field()
    # english: int = field()

    def __attrs_post_init__(self):
        print('attrs_post_init')
    # def __init__(self, name, math, chinese, english):
    #     self.name = name
    #     self.math = math
    #     self.chinese = chinese
    #     self.english = english

    def __repr__(self):
        return f"Student: {self.name}, math: {self.math}, chinese: {self.chinese}, english: {self.english}"

stu = Student('Tom', 90, 80, 70)
pp(attrs.asdict(stu))
pp(stu)
        

attrs_pre_init
attrs_post_init
custome init
{'chinese': 80, 'english': 70, 'math': 90, 'name': 'Tom'}
Student: Tom, math: 90, chinese: 80, english: 70


In [7]:
@define(slots=False)
class DataDes:
    _score: field(default=0)
    # def __init__(self, score=0):
    #     self._score = score
    def __set__(self, instance, value):
        self._score = value
    def __get__(self, instance, owner):
        print("访问数据描述符里的__get__")
        return self._score

@define(slots=False)
class NonDataDes:
    _score: field(default=0)
    # def __init__(self, score=0):
    #     self._score = score
    def __get__(self, instance, owner):
        print("访问非数据描述符里的__get__")
        return self._score


@define(slots=False)
class Student:
    _math = DataDes(0)
    _chinese = NonDataDes(0)

    _name: str
    _math: int
    _chinese: int
    
    def __getattribute__(self, item):
        print(f"调用__getattribute__{item}")
        return super().__getattribute__(item)
    
    def __repr__(self):
        return f"Student: {self._name}, math: {self._math}, chinese: {self._chinese}"
    
stu = Student(chinese = 90, math = 80, name='Tom')
pp(attrs.asdict(stu))

# pp(attrs.asdict(stu))
# pp(stu._math)
# pp(stu._chinese)
# print(stu)




访问数据描述符里的__get__
访问非数据描述符里的__get__
调用__getattribute____class__
调用__getattribute___name
调用__getattribute___math
调用__getattribute___chinese
{'_chinese': 90, '_math': 80, '_name': 'Tom'}


In [8]:
a = {
    "1": {
        "1": {
            "1": []}},
    "2": {
        "1": [],
        "2": {
            "1": []
        }
    }
}
def give_value(data):
    for k, v in data.items():
        # print(k, v)
        if isinstance(v, list):
            print("Yes")
            data[k] = 1
            # v = 1
        else:
            print(v)
            give_value(v)
give_value(a)
print(a)

{'1': {'1': []}}
{'1': []}
Yes
{'1': [], '2': {'1': []}}
Yes
{'1': []}
Yes
{'1': {'1': {'1': 1}}, '2': {'1': 1, '2': {'1': 1}}}


In [9]:
class Data:
    def __init__(self):
        pass


idx = 0
def create_node(data, value):
    data.__setattr__(str(value), value)

data = Data()
setattr(data, "1", Data())
setattr(getattr(data,"1"), "1", Data())

data.__setattr__("2", Data())
data.__getattribute__("2").__setattr__("3", 15)
 
print(data)
print(data.__dict__)
create_node(data.__getattribute__("2"), 15)

print(data.__getattribute__("2").__dict__)

{'1': <__main__.Data object at 0x000001D1969B0EB0>, '2': <__main__.Data object at 0x000001D196ABCD90>}
{'3': 15, '15': 15}


In [10]:
from dataclasses import dataclass
@dataclass
class Data:
    

SyntaxError: unexpected EOF while parsing (891763077.py, line 4)

In [ ]:
class Test:
    def __init__(self):
        pass
    def __getitem__(self, key):
        if isinstance(key, int):
            return self.List[key]
        elif isinstance(key, str):
            return self.__dict__[key]
    def __setitem__(self, key, value):
        if isinstance(key, str):
            if 'Dict' not in self.__dict__:
                self.__dict__['Dict'] = {}
            self.__dict__['Dict'][key] = value
        if isinstance(key, int):
            if 'List' not in self.__dict__:
                self.__dict__['List'] = [0, 0, 0]
            self.__dict__['List'][key] = value

t = Test()
t['a'] = 1
t[0] = 2

print(t.__dict__)




{'Dict': {'a': 1}, 'List': [2, 0, 0]}


In [ ]:

from dataclasses import dataclass
# @dataclass
class OD(dict):
    def __init__(self, *args, **kwargs):
        # print(self.__dict__)
        # print(dir(self))
        super().__init__(*args, **kwargs)
        # print(self.__dict__)
        # print(dir(self))
        # self.__dict__ = self
    # def __getattr__(self, name):
    #     value = self[name]
    #     print(value)
    #     if isinstance(value, dict):
    #         value = OD(value)
    #         print("    {0}", value)
    #     return value

od = OD(asf={'a': [1, 2, 3], 'b': 2}, b=2)
od.asf = OD(s=1)
# od.asf = OD(a=1, b=2)
# print(od.b)
print(od.asf)
print(od)
print(od.__dict__)
# print(od.__getattr__('a'))
# print(dict.__dict__)


{'s': 1}
{'asf': {'a': [1, 2, 3], 'b': 2}, 'b': 2}
{'asf': {'s': 1}}


In [ ]:

class OD(dict):
    def __init__(self, *args, **kwargs):
        pass

od = OD(asf=1, b=2)
print(od)

{}


In [ ]:
class Data:
    def __init__(self, t):
        if isinstance(t, str):
            if t in ('list', 'dict'):
                self._type = {'list': list, 'dict': dict}[t]
            else:
                raise ValueError("type string must be 'list' or 'dict'")
        elif isinstance(t, type):
            if t in (list, dict):
                self._type = t
            else:
                raise ValueError("type must be list or dict")
        else:
            raise ValueError("type must be string or type")
        
    def __setitem__(self, index, value):
        self.__dict__[str(index)] = value
        # self.__settattr__(str(index), value)
    def __getitem__(self, index):
        return self.__dict__[str(index)]
        # return self.__getattr__(str(index))
    def __setattr__(self, index, value):
        self.__dict__[str(index)] = value
    def __getattr__(self, index, value):
        return self.__dict__[str(index)]
    @property
    def value(self):
        keys = set(self.__dict__.keys())
        keys.remove('_type')
        if self._type == list:
            sorted_keys = sorted([int(i) for i in keys])
            rlt = [self.__dict__[str(i)] for i in sorted_keys]
            return rlt
        if self._type == dict:
            rlt = {i: self.__dict__[i] for i in keys}
            return rlt

            
        


data = Data(list)
data[1] = "abc"
data[2]= "def"
print(data.__dict__)
print(data.value)

data2 = Data(dict)
data2[1] = "qwe"
data2[2] = "asd"
print(data2.__dict__)
print(data2.value)
    
        
    

{'_type': <class 'list'>, '1': 'abc', '2': 'def'}
['abc', 'def']
{'_type': <class 'dict'>, '1': 'qwe', '2': 'asd'}
{'1': 'qwe', '2': 'asd'}


In [ ]:
import xlwings as xw
import re
wb_path = r"D:\GIT\Github\TinyTool\xlsx\test.xlsx"
ws_name = "Test2"
class Range:
    pass


app = xw.App(visible=True, add_book=False)
with app.books.open(wb_path) as wb:
    ws = wb.sheets[ws_name]
    rng = ws.range(1, 3).merge_area.address
    print(rng)
    # print(dir(rng))
    # print(f"rows: {rng.rows}")
    # print(f"columns: {rng.columns}")
    # print(f"address: {rng.address}")
    # print(f"column_width: {rng.column_width}")
    # print(f"row_height: {rng.row_height}")
    # print(f"height: {rng.height}")
    # print(f"width: {rng.width}")
    # print(f"merge_area: {rng.merge_area}")







app.quit()

$C$1:$J$1


In [ ]:
pr = True
def print_return(pr=True):
    def deco(func):
        def wrapper(*args, **kwargs):
            rlt = func(*args, **kwargs)
            if pr :
                print(rlt)
            return rlt
        return wrapper
    return deco
        

# @print_return(pr)
# def is_digit(s):
#     return s.isdigit()

import re
def col_to_num(col_str):
    """ Convert base26 column string to number. """
    expn = 0
    col_num = 0
    for char in reversed(col_str):
        col_num += (ord(char) - ord('A') + 1) * (26 ** expn)
        expn += 1
    return col_num

@print_return(pr)
def convert_address(address):
    reg = re.compile(r"([a-zA-Z]+|[0-9]+)")
    # reg_num = re.compile(r"[0-9]+")
    strs = reg.findall(address)
    print(strs)
    rlts = []
    for s in strs:
        if s.isdigit():
            rlts.append(int(s))
        else:
            rlts.append(col_to_num(s.upper()))
    if len(rlts) == 1:
        result = rlts[0]
    elif len(rlts) == 2:
        result = (rlts[0], rlts[1])
    elif len(rlts) ==4:
        result = ((rlts[0], rlts[1]), (rlts[2], rlts[3]))

    return result


a = "$CA$1:$J$1"
b = "ah"
c = "a12"
# print(convert_address(a))
# print(convert_address(b))
# print("12".isdigit())
# is_digit("12")
convert_address(a)
convert_address(b)
convert_address(c)




['CA', '1', 'J', '1']
((79, 1), (10, 1))
['ah']
34
['a', '12']
(1, 12)


(1, 12)

In [100]:
    
class Data:
    def __init__(self, t):
        if isinstance(t, str):
            if t in ('list', 'dict'):
                self._type = {'list': list, 'dict': dict}[t]
            else:
                raise ValueError("type string must be 'list' or 'dict'")
        elif isinstance(t, type):
            if t in (list, dict):
                self._type = t
            else:
                raise ValueError("type must be list or dict")
        else:
            raise ValueError("type must be string or type")
        
    def __setitem__(self, index, value):
        self.__dict__[str(index)] = value
        # self.__settattr__(str(index), value)
    def __getitem__(self, index):
        return self.__dict__[str(index)]
        # return self.__getattr__(str(index))
    def __setattr__(self, index, value):
        self.__dict__[str(index)] = value
    def __getattr__(self, index, value):
        return self.__dict__[str(index)]

    
    # @classmethod
    # @property
    # def value(self):
    #     keys = set(self.__dict__.keys())
    #     keys.remove('_type')
    #     # print(keys)
    #     if self._type == list:
    #         # print("list")
    #         sorted_keys = sorted([int(i) for i in keys])
    #         for i in sorted_keys:
    #             if type(self.__dict__[str(i)]) is Data:
    #                 rlt = [self.__dict__[str(i)].value() for i in sorted_keys]
    #             else:
    #                 rlt = [self.__dict__[str(i)] for i in sorted_keys]
    #     if self._type == dict:
    #         # print("dict")
    #         for i in keys:
    #             if type(self.__dict__[i]) is Data:
    #                 rlt = {i: self.__dict__[i].value() for i in keys}
    #             else:
    #                 rlt = {i: self.__dict__[i] for i in keys}
    #     return rlt

def render_value(obj):
    if type(obj) is Data:
        keys = set(obj.__dict__.keys())
        keys.remove('_type')
        # print(keys)
        if obj._type == list:
            sorted_keys = sorted([int(i) for i in keys])
            for i in sorted_keys:
                # print("{0} {1}".format(obj.__dict__, type(obj.__dict__[str(i)])))
                if type(obj.__dict__[str(i)]) is Data:
                    rlt = [render_value(obj.__dict__[str(i)]) for i in sorted_keys]
                else:
                    rlt = [obj[str(i)] for i in sorted_keys]
        if obj._type == dict:
            for i in keys:
                # print("{0} {1}".format(obj.__dict__, type(obj.__dict__[i])))
                if type(obj.__dict__[i]) is Data:
                    rlt = {i: render_value(obj.__dict__[i]) for i in keys}
                else:
                    rlt = {i: obj[i] for i in keys}
    else:
        rlt = obj
    return rlt

data = Data('dict')
data.a = 1
data.b = Data('list')
data.c = "abcd"
data.b[1] = Data('dict')
data.b[1].a = 1
data.b[0] = Data('dict')
data.b[0].a = 2

# print(data)
r = render_value(data)
print(r)

{'c': 'abcd', 'b': <__main__.Data object at 0x000001D1983FF7F0>, 'a': 1}


In [ ]:
class Node:
    def __init__(self, value) -> None:
        self._value = value
        self._children = []
    def __repr__(self) -> str:
        return f"Node({self._value})"
    def add_child(self, node):
        self._children.append(node)
    def __iter__(self):
        return iter(self._children)
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first()

root = Node(0)
c1 = Node(1)
c2 = Node(2)
root.add_child(c1)
root.add_child(c2)
c1.add_child(Node(3))
c2.add_child(Node(4))

for i in root.depth_first():
    print(i)


Node(0)
Node(1)
Node(3)
Node(2)
Node(4)


In [18]:
a = (1, 2)
c = [[1, 2]]
b = tuple([a])
c = tuple(c)
print(len(b))
print(c)


1
([1, 2],)


In [ ]:

class Countdown:
    def __init__(self, step) -> None:
        print("start init")
        self.step = step
        print("init done\n")
    def __next__(self):
        print("start next with step: {0}".format(self.step))
        if self.step > 5:
            raise StopIteration
        self.step += 1
        print("end next with step: {0}\n".format(self.step))
        return self
    def __iter__(self):
        return self#.__next__()

# c = Countdown(0)
# next(c)
# for i in c:
#     print(i)
# for idx, i in enumerate(c):
#     print(idx, i.step)

# for i in range(12):
#     # print(c.step)
#     print(i)
#     print("step{0}".format(c.step))
#     next(c)

for idx, i in enumerate(Countdown(0)):
    print("idx {0}".format(idx))
    # print(i.step)
# print(list(Countdown(0)))




In [ ]:
def func():
    print("start func")
    while True:
        res = yield  4
        print("res: {0}".format(res))


f = func()
print("###")
print(next(f))
print("===")
print(next(f))
print("+++")
print(next(f))
print("---")
print(next(f))

In [86]:
class C:
    def __init__(self, step=0):
        self.step = step
        print("init called\n")
        pass
    def __iter__(self):
        print("iter called\n")
        return self
    def __next__(self):
        print("next called")
        print("self.step before += 1: {0}".format(self.step))
        if self.step > 5:
            raise StopIteration
        self.step += 1
        print("self.step after += 1: {0}\n".format(self.step))
        # yield self
        return self
        print('yield done')

c = C()
print("===")
for idx, i in enumerate(c):
    print(idx)
    print(i.step)
    # next(i)
    # print(i.step)



init called

===
iter called

next called
self.step before += 1: 0
self.step after += 1: 1

0
1
next called
self.step before += 1: 1
self.step after += 1: 2

1
2
next called
self.step before += 1: 2
self.step after += 1: 3

2
3
next called
self.step before += 1: 3
self.step after += 1: 4

3
4
next called
self.step before += 1: 4
self.step after += 1: 5

4
5
next called
self.step before += 1: 5
self.step after += 1: 6

5
6
next called
self.step before += 1: 6


In [95]:
class C1:
    v1 = "i am class variable"
print(C1.v1)
print(id(C1.v1))
c1 = C1()
print(c1.v1)
print(id(c1.v1))
print("====")
C1.v1 = "i am still class vairalbe"
print(C1.v1)
print(id(C1.v1))
print(c1.v1)
print(id(c1.v1))
print("====")
c1.v1 = "i am instance variable"
print(C1.v1)
print(id(C1.v1))
print(c1.v1)
print(id(c1.v1))
print("====")
C1.v1 = "Are you instance variable?"
print(C1.v1)
print(id(C1.v1))
print(c1.v1)
print(id(c1.v1))




i am class variable
1999709598784
i am class variable
1999709598784
====
i am still class vairalbe
1999708935472
i am still class vairalbe
1999708935472
====
i am still class vairalbe
1999708935472
i am instance variable
1999712275952
====
Are you instance variable?
1999712273472
i am instance variable
1999712275952
